In [ ]:
from langgraph.graph import StateGraph,START,END
from langchain_groq import ChatGroq

from typing import TypedDict
from dotenv import load_dotenv

In [96]:
load_dotenv()

True

In [ ]:
model = ChatGroq(
    model = "lama-3.1-8b-instant",
    temperature = 0
)

In [98]:
class BatsmanState(TypedDict):

    runs: int
    balls : int
    fours : int
    sixes: int

    sr : float
    bpb : float
    boundary_percent : float
    summary : str

In [99]:
def calculate_sr(state: BatsmanState):

    sr = (state['runs']/state['balls'])*100

    #state['sr'] =  sr

    return {'sr':sr}

In [100]:
def calculate_bpb(state: BatsmanState):

    bpb = state['balls']/(state['fours']+state['sixes'])

    #state['bpb'] = bpb

    return {'bpb':bpb}

In [101]:
def calculate_boundary_percent(state: BatsmanState):

    boundary_percent = (((state['fours']*4 + state['sixes']*6))/state['runs'])*100

   # state['boundary_percent'] = boundary_percent

    return {'boundary_percent':boundary_percent}

In [102]:
def summary(state: BatsmanState):
    summary_text = (
        f"Strike Rate - {state['sr']}\n"
        f"Balls Per Boundary - {state['bpb']}\n"
        f"Boundary Percentage - {state['boundary_percent']}"
    )
    return {'summary': summary_text}


In [103]:
graph = StateGraph(BatsmanState)


graph.add_node('calculate_sr',calculate_sr)
graph.add_node('calculate_bpb',calculate_bpb)
graph.add_node('calculate_boundary_percent',calculate_boundary_percent)
graph.add_node('summary',summary)


graph.add_edge(START,'calculate_sr')
graph.add_edge(START,'calculate_bpb')
graph.add_edge(START,'calculate_boundary_percent')
graph.add_edge('calculate_sr','summary')
graph.add_edge('calculate_bpb','summary')
graph.add_edge('calculate_boundary_percent','summary')
graph.add_edge('summary',END)

workflow = graph.compile()

In [104]:
initial_state ={
    'runs':100,
    'balls': 50,
    'fours':6,
    'sixes':4
    }

final_state = workflow.invoke(initial_state)

print(final_state)

{'runs': 100, 'balls': 50, 'fours': 6, 'sixes': 4, 'sr': 200.0, 'bpb': 5.0, 'boundary_percent': 48.0, 'summary': 'Strike Rate - 200.0\nBalls Per Boundary - 5.0\nBoundary Percentage - 48.0'}
